In [20]:
# !pip install evaluate
# !pip install seqeval

In [22]:
# !pip install accelerate -U
# !pip install transformers[torch]

In [2]:
import sys
sys.path.append("..")
from helper import align_labels
import helper
import collections

## Dataset

### Read the Train Dataset (Sentences and Labels)

In [3]:
# Open the Data of Sentences
with open('subtask2_train.data.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()


# Open the label data
with open('subtask2_train.labels.txt', 'r', encoding='utf-8') as file:
    labels = file.readlines()

### Convert the Dataset into a pandas dataframe

In [4]:
import pandas as pd

dataset = pd.DataFrame(list(zip(sentences, labels)), columns=['sentences', 'labels'])
# split sentences and labels
dataset['sentences'] = dataset['sentences'].apply(lambda row: row.split())
dataset['labels'] = dataset['labels'].apply(lambda row: row.split())

### Unique labels

In [5]:
unique_labels = []

for line in labels:
    labels_list = line.split()
    for label in labels_list:
        if label not in unique_labels:
            unique_labels.append(label)

uniqueLabel_to_ID = {unique_label: ID for ID, unique_label in enumerate(unique_labels)}
ID_to_uniqueLable = {ID: unique_label for ID, unique_label in enumerate(unique_labels)}

### Map each label to its ID

In [6]:
dataset['IDs'] = dataset['labels'].apply(lambda row: [uniqueLabel_to_ID.get(label) for label in row])

# Prepare the train and validation dataset

In [7]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(dataset, test_size=0.25, random_state=42)

In [8]:
len(train), len(validation)

(1764, 589)

## Tokenize and convet the labels from tokenized into IDs

In [9]:
from transformers import BertTokenizerFast, AutoTokenizer

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
def tokenization(input_data):
    tokenized_train_inputs = tokenizer(
        input_data,
        return_tensors="pt",
        truncation=True,
        is_split_into_words=True,
        padding="do_not_pad",
        max_length=512,
    )

    return tokenized_train_inputs

### Trian Tokenization

In [11]:
train['tokenized'] = train.apply(lambda row: helper.tokenized_align_labels(tokenization(row['sentences']), row['IDs']), axis=1)

### Validation Tokenization

In [12]:
validation['tokenized'] = validation.apply(lambda row: helper.tokenized_align_labels(tokenization(row['sentences']), row['IDs']), axis=1)

### Prepare Train dataset for fine-tuning

In [13]:
Train_dataset = train.tokenized.apply(
    lambda x: {
        k: v[0]
        if type(v) is not list
        else torch.tensor(v)
        for k, v in x.items()}).to_list()

### Prepare validation dataset for fine-tuning

In [14]:
Val_dataset = validation.tokenized.apply(
    lambda x: {
        k: v[0]
        if type(v) is not list
        else torch.tensor(v)
        for k, v in x.items()}).to_list()

## Load pretrained model

In [15]:
from transformers import AutoModelForTokenClassification
### todo guck mal nach cased und uncased ob es mit cased besser funktioniert oder mit uncased
model_name = "bert-base-uncased"
model = AutoModelForTokenClassification.from_pretrained(model_name, id2label=ID_to_uniqueLable, num_labels=len(unique_labels))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Evaluation

In [44]:
import evaluate
import numpy as np
metric = evaluate.load('seqeval')


def compute_metrics(eval_preds, label_class=unique_labels):
    logits, labels = eval_preds
    # becase the logics and probabilities both are in the same order, we don't need to aply softmax here
    predictions = np.argmax(logits, axis=-1)
    # now we need to remove all the values, where the label is -100
    # before passing to metric.compute we should have these inputs as a list
    true_labels = [[label_class[l] for l in label if l != -100]
                   for label in labels]

    true_predictions = [[label_class[p] for p,l in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    support_all = 0
    f1_weighted = {}
    label_values = {}
    entity_labels = list({l[2:] for l in label_class if l != 'O'})
    for label in entity_labels:
      label_metrics = all_metrics.get(label, dict())
      for metric_label, value in label_metrics.items():
        if metric_label == 'number':
          support_all += value
        if metric_label == 'f1':
          f1_weighted[label] = value * label_metrics['number']
        label_values[f"{metric_label}_{label}"] = value
      if label_metrics:
        del all_metrics[label]
    f1_macro_weighted = 0.
    if support_all != 0:
      for f1 in f1_weighted.values():
        f1_macro_weighted += f1 / support_all
    all_metrics[f"f1_macro_weighted"] = f1_macro_weighted
    all_metrics |= label_values
    return all_metrics

# Parameter optimization

## Define, perform the hyperparameter search

In [42]:
# !pip install transformers optuna datasets

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments, BertForTokenClassification
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)
def model_init():
    return BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_labels), return_dict=True)



def objective(trial):
    learning_rate = trial.suggest_float('learning_rate',  1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16, 32])
    epochs = trial.suggest_int("num_train_epochs", 3, 7)
    weight_decay = trial.suggest_float("weight_decay",  1e-5, 0.1, log=True)

    training_args = TrainingArguments(
        output_dir="./output",
        eval_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=100,
        warmup_ratio=0.1,
        gradient_accumulation_steps=2,
        max_grad_norm=1.0
    )



    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=Train_dataset,
        eval_dataset=Val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    eval_result = trainer.evaluate()
    overall_f1 = eval_result['eval_f1_macro_weighted']
    return overall_f1

study = optuna.create_study(direction='maximize')
assert len(study.trials) == 0
study.optimize(objective, n_trials=20)

if len(study.trials) == 0 or all([t.state != optuna.trial.TrialState.COMPLETE for t in study.trials]):
    print("No trials are completed yet.")
else:
  print("Best trial:")
  trial = study.best_trial
  print(trial.values)
  print("Best hyperparameters: {}".format(trial.params))


[I 2024-07-16 10:49:33,407] A new study created in memory with name: no-name-260f22a8-439e-47c8-8d9f-2925842f12bb
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,F1 Macro Weighted,Precision Version,Recall Version,F1 Version,Number Version,Precision Alternativename,Recall Alternativename,F1 Alternativename,Number Alternativename,Precision Extension,Recall Extension,F1 Extension,Number Extension,Precision Citation,Recall Citation,F1 Citation,Number Citation,Precision License,Recall License,F1 License,Number License,Precision Developer,Recall Developer,F1 Developer,Number Developer,Precision Release,Recall Release,F1 Release,Number Release,Precision Url,Recall Url,F1 Url,Number Url,Precision Abbreviation,Recall Abbreviation,F1 Abbreviation,Number Abbreviation
0,No log,0.142776,0.745736,0.677465,0.709963,0.969494,0.645675,0.884488,0.940351,0.911565,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.696629,0.953846,0.805195,130,0.000000,0.000000,0.000000,9,0.162791,0.090323,0.116183,155,0.000000,0.000000,0.000000,16,0.961538,0.937500,0.949367,80,0.000000,0.000000,0.000000,15
2,0.533500,0.072795,0.791557,0.845070,0.817439,0.981672,0.788755,0.873016,0.964912,0.916667,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.767123,0.861538,0.811594,130,0.000000,0.000000,0.000000,9,0.649038,0.870968,0.743802,155,0.666667,0.125000,0.210526,16,0.915663,0.950000,0.932515,80,0.000000,0.000000,0.000000,15


[I 2024-07-16 10:51:14,671] Trial 0 finished with value: 0.7887551417206412 and parameters: {'learning_rate': 2.4612471273961025e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 2.61958694318397e-05}. Best is trial 0 with value: 0.7887551417206412.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,F1 Macro Weighted,Precision Version,Recall Version,F1 Version,Number Version,Precision Alternativename,Recall Alternativename,F1 Alternativename,Number Alternativename,Precision Extension,Recall Extension,F1 Extension,Number Extension,Precision Citation,Recall Citation,F1 Citation,Number Citation,Precision License,Recall License,F1 License,Number License,Precision Developer,Recall Developer,F1 Developer,Number Developer,Precision Release,Recall Release,F1 Release,Number Release,Precision Url,Recall Url,F1 Url,Number Url,Precision Abbreviation,Recall Abbreviation,F1 Abbreviation,Number Abbreviation
0,1.118800,0.140258,0.739264,0.678873,0.707783,0.970724,0.645753,0.887417,0.940351,0.913118,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.696629,0.953846,0.805195,130,0.000000,0.000000,0.000000,9,0.152174,0.090323,0.113360,155,0.000000,0.000000,0.000000,16,0.950000,0.950000,0.950000,80,0.000000,0.000000,0.000000,15
2,0.071100,0.063725,0.827397,0.850704,0.838889,0.983701,0.816993,0.895425,0.961404,0.927242,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.805970,0.830769,0.818182,130,0.500000,0.333333,0.400000,9,0.710526,0.870968,0.782609,155,0.571429,0.500000,0.533333,16,0.950000,0.950000,0.950000,80,0.000000,0.000000,0.000000,15
4,0.046500,0.057103,0.797665,0.866197,0.830520,0.985731,0.818275,0.901316,0.961404,0.930390,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.793103,0.884615,0.836364,130,0.333333,0.333333,0.333333,9,0.700508,0.890323,0.784091,155,0.360000,0.562500,0.439024,16,0.938272,0.950000,0.944099,80,0.000000,0.000000,0.000000,15


[I 2024-07-16 10:54:10,085] Trial 1 finished with value: 0.8182746401597264 and parameters: {'learning_rate': 1.109728005766158e-05, 'batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.00039908643992766346}. Best is trial 1 with value: 0.8182746401597264.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,F1 Macro Weighted,Precision Version,Recall Version,F1 Version,Number Version,Precision Alternativename,Recall Alternativename,F1 Alternativename,Number Alternativename,Precision Extension,Recall Extension,F1 Extension,Number Extension,Precision Citation,Recall Citation,F1 Citation,Number Citation,Precision License,Recall License,F1 License,Number License,Precision Developer,Recall Developer,F1 Developer,Number Developer,Precision Release,Recall Release,F1 Release,Number Release,Precision Url,Recall Url,F1 Url,Number Url,Precision Abbreviation,Recall Abbreviation,F1 Abbreviation,Number Abbreviation
0,No log,0.133467,0.747692,0.684507,0.714706,0.970847,0.660264,0.903780,0.922807,0.913194,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.696629,0.953846,0.805195,130,0.000000,0.000000,0.000000,9,0.227723,0.148387,0.179687,155,0.000000,0.000000,0.000000,16,0.950000,0.950000,0.950000,80,0.000000,0.000000,0.000000,15
2,0.581600,0.053116,0.840108,0.873239,0.856354,0.985792,0.853232,0.915825,0.954386,0.934708,285,0.000000,0.000000,0.000000,10,1.000000,0.300000,0.461538,10,0.835938,0.823077,0.829457,130,0.500000,0.444444,0.470588,9,0.787709,0.909677,0.844311,155,0.409091,0.562500,0.473684,16,0.950000,0.950000,0.950000,80,0.571429,0.533333,0.551724,15
4,0.042600,0.050768,0.852785,0.905634,0.878415,0.988191,0.878902,0.925926,0.964912,0.945017,285,0.714286,0.500000,0.588235,10,1.000000,0.700000,0.823529,10,0.824818,0.869231,0.846442,130,0.444444,0.444444,0.444444,9,0.786517,0.903226,0.840841,155,0.631579,0.750000,0.685714,16,0.938272,0.950000,0.944099,80,0.578947,0.733333,0.647059,15
5,0.021200,0.050987,0.853403,0.918310,0.884668,0.988437,0.886399,0.923077,0.968421,0.945205,285,0.857143,0.600000,0.705882,10,1.000000,0.700000,0.823529,10,0.825175,0.907692,0.864469,130,0.500000,0.555556,0.526316,9,0.804598,0.903226,0.851064,155,0.545455,0.750000,0.631579,16,0.938272,0.950000,0.944099,80,0.571429,0.800000,0.666667,15


[I 2024-07-16 11:14:21,029] Trial 7 finished with value: 0.8863994096656127 and parameters: {'learning_rate': 3.340689577220468e-05, 'batch_size': 16, 'num_train_epochs': 6, 'weight_decay': 0.050388269317350444}. Best is trial 4 with value: 0.8896188587836443.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,F1 Macro Weighted,Precision Version,Recall Version,F1 Version,Number Version,Precision Alternativename,Recall Alternativename,F1 Alternativename,Number Alternativename,Precision Extension,Recall Extension,F1 Extension,Number Extension,Precision Citation,Recall Citation,F1 Citation,Number Citation,Precision License,Recall License,F1 License,Number License,Precision Developer,Recall Developer,F1 Developer,Number Developer,Precision Release,Recall Release,F1 Release,Number Release,Precision Url,Recall Url,F1 Url,Number Url,Precision Abbreviation,Recall Abbreviation,F1 Abbreviation,Number Abbreviation
0,No log,0.216122,0.945736,0.343662,0.504132,0.952642,0.403081,0.947137,0.754386,0.839844,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.935484,0.223077,0.360248,130,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,155,0.000000,0.000000,0.000000,16,0.000000,0.000000,0.000000,80,0.000000,0.000000,0.000000,15
2,0.671000,0.093758,0.775839,0.814085,0.794502,0.979458,0.762225,0.869427,0.957895,0.911519,285,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,10,0.727273,0.861538,0.788732,130,0.000000,0.000000,0.000000,9,0.596939,0.754839,0.666667,155,0.000000,0.000000,0.000000,16,0.938272,0.950000,0.944099,80,0.000000,0.000000,0.000000,15


[I 2024-07-16 11:16:02,908] Trial 8 finished with value: 0.762224600747695 and parameters: {'learning_rate': 1.4293914088113572e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.0006987185712216486}. Best is trial 4 with value: 0.8896188587836443.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


## Retrain with best parameters

In [19]:
V

NameError: name 'V' is not defined

In [22]:
trial.params, trial.value

({'learning_rate': 4.03137219824488e-05,
  'batch_size': 4,
  'num_train_epochs': 5,
  'weight_decay': 2.6675525585646302e-05},
 0.8789546079779919)